In [1]:
import os

import pandas as pd
from data_loader import label_str2int_dict

rationale_names = ['td','itc','img','cs']
print(os.getcwd())
os.chdir('/media/shared_d/lyq/DataSet/FakeNews/weibo_dataset')
print(os.getcwd())


/home/lyq/PycharmProjects/QwenVLRationaleGenerate/data/weibo
/media/shared_d/lyq/DataSet/FakeNews/weibo_dataset


In [2]:
import ast

weibo_df = pd.read_csv('weibo.csv')
weibo_df = weibo_df.rename(columns={
    'id':'source_id',
    'text':'content',
})

def get_image_id(image_file_name_list):
    return ast.literal_eval(image_file_name_list)[0].split('/')[-1].split('.')[0]


weibo_df['content'] = weibo_df.apply(lambda x : f'{x["content"]} --发布来源：{x["release_source"]}', axis=1)
weibo_df['label'] = weibo_df['label'].apply(lambda x : label_str2int_dict[x])
weibo_df['image_id'] = weibo_df['available_image_paths'].apply(lambda x : get_image_id(x))

weibo_df = weibo_df.drop(columns=['available_image_paths','release_source'])

weibo_df

,source_id,content,split,label,image_id
0,3511947309647762,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,train,1,a71ac854gw1dytin2zmk9j
1,3576100079039606,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,train,1,3ba161e7jw1e4i6j4ep61j211n0r4juz
2,3899073935617462,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",train,1,006ajvCngw1ex4jjl2pwlj30c8096wf7
3,3584521306131914,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",train,1,6112c76ajw1e591p39mpej20hw094mxh
4,3553661986467439,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,train,1,a5404163jw1e2iljwbrztj
...,...,...,...,...,...
7848,3896157270699471,【空城记：刘小东与鄂尔多斯】#正午故事#鄂尔多斯是个比较敏感的城市。其实政府也比想象的聪明，...,test,0,005EHRy1jw1ewv1azxwh1j30rs0lljzm
7849,3895644894378737,【日本人是怎么拍黑帮电影的？】Tim Gallo出生于俄国，17岁后迁往日本生活。通过用镜头...,test,0,005EHRy1jw1ewtlhknf30j30jg0czq4y
7850,3893257346088858,#跟着面叔看世界#第一站，日本东京筑地市场。你现在到哪儿了？不会还堵在路上吧？一起来晒晒旅途...,test,0,005EHRy1jw1ewlzj6oenaj30qo0zkdm5
7851,3892828281041275,【现场图·广西柳城县连续发生多起爆炸】据@南宁电视台 ：网友称，现场听到几声巨响，车站，市场...,test,0,61e04755jw1ewkm8o7fyyj20c80gb0u2


In [3]:
data_set_len = weibo_df.shape[0]

def findNoRationaleNews(df,r_df):
    return set(df['source_id'].tolist()) - set(r_df['source_id'].tolist())

for rationale_name in rationale_names:
    r_df = pd.read_csv(f'{rationale_name}.csv')
    r_df = r_df.dropna(subset=['authenticity','reason'])
    r_df['authenticity'] = r_df['authenticity'].apply(lambda x : label_str2int_dict[x])
    if len(findNoRationaleNews(weibo_df,r_df)) > 0:
        raise Exception(f'{rationale_name} len : {r_df.shape[0]} != {data_set_len} , No Rationale id : {findNoRationaleNews(weibo_df,r_df)}')
    weibo_df = weibo_df.merge(r_df,how='left',on='source_id')
    weibo_df = weibo_df.rename(columns={
        'authenticity':f'{rationale_name}_pred',
        'reason': f'{rationale_name}_rationale'
    })
    weibo_df[f'{rationale_name}_acc'] = weibo_df.apply(lambda x : int(x[f'{rationale_name}_pred'] == x['label']), axis=1)


weibo_df

,source_id,content,split,label,image_id,td_pred,td_rationale,td_acc,itc_pred,itc_rationale,itc_acc,img_pred,img_rationale,img_acc,cs_pred,cs_rationale,cs_acc
0,3511947309647762,震惊，转发求证：【想都不敢想 ，在美国一桶金龙鱼食用油只要8元人民币】 一桶食用油相当于中国...,train,1,a71ac854gw1dytin2zmk9j,1,1. **价格对比不合理**：该消息声称在美国沃尔玛，一桶金龙鱼食用油的价格低至1.3美元（...,1,1,1. **事件**：图片展示的是金龙鱼食用油的产品图，这与新闻中提到的美国沃尔玛感恩节时的价...,1,0,1. 图片来源\n- **水印或标识**：图片中有“@地球超级搞笑”、“@地球都要笑了”等水...,0,1,- 合理性：价格差异：中国的食用油价格受到多种因素的影响，包括生产成本、运输费用、税收政策等...,1
1,3576100079039606,【法院无底线】湖南长沙一位小朋友上学路上捡到3万元，原地不动等失主，结果被人冒领。不知情的孩...,train,1,3ba161e7jw1e4i6j4ep61j211n0r4juz,1,1. **来源可信度**：该消息来源于个人微博账号“瀟湘墨人/@3千里”，并通过手机客户端发...,1,1,1. **事件**：新闻报道了一个小朋友捡到钱后被冒领，然后被真失主告上法庭的故事。然而，图...,1,0,1. 图片来源\n- **水印或标识**：图片中有“@胶东县令”和“weibo.com/u/...,0,1,合理性：虽然孩子捡到钱并等待失主的行为是值得赞扬的好事，但最终被法院判决赔偿的情况显得不合常...,1
2,3899073935617462,"转发:我校需要小孩的衣服,新旧不限!西藏阿里地区是世界海拔最高的地区请问周围有没有四到十岁孩...",train,1,006ajvCngw1ex4jjl2pwlj30c8096wf7,1,1. 发布来源：信息中提到的发布来源是“vivo X5Pro”，这是一个手机型号，而不是一个...,1,1,1. **事件**：新闻中提到的是关于西藏阿里地区札达县曲松乡中心小学需要小孩衣服的求助信息...,1,0,1. 图片来源\n- **水印或标识**：图片中有“@微然”的水印，表明这张图片可能来源于某...,0,0,合理性：请求捐赠衣物的信息通常是真实的，特别是当涉及到偏远地区的学校时。这些地方由于地理位置...,0
3,3584521306131914,"立刻检查一下你家里的牙膏，如果是黑色条马上扔掉！ 大家买膏请留心,买牙膏时注意牙膏管反面 底...",train,1,6112c76ajw1e591p39mpej20hw094mxh,1,1. 该消息声称牙膏底部的颜色条代表不同的成分，但这种说法并没有科学依据或官方认证。\n2....,1,1,1. **事故**： 图片显示了两支牙膏，一支是黑色的，另一支是蓝色的。这与新闻中提到的牙膏...,1,0,1. 图片来源\n- **水印或标识**：图片中有“凤凰吴氏制茶”的水印，表明这张图片可能来...,0,1,合理性：牙膏底部的颜色条实际上是一种印刷标记，用于帮助机器在生产过程中识别包装的方向，而不是...,1
4,3553661986467439,快快转发，急急急！中央电视台《焦点访谈》已经播出，可口可乐承认旗下(果粒橙)含有美国禁用农药...,train,1,a5404163jw1e2iljwbrztj,1,\n1. **紧急呼吁**：开头的“快快转发，急急急！”是一种常见的谣言传播手法，目的是引起...,1,1,1. **事件**：新闻内容提到可口可乐旗下产品“果粒橙”含有多菌灵，这是一种在美国被禁用的...,1,0,1. 图片来源\n- **水印或标识**：图片中没有明显的水印或标识，无法直接判断其来源。\...,0,1,- 似是而非：这条消息中的内容具有一定的误导性。首先，如果中央电视台《焦点访谈》确实报道了这...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7848,3896157270699471,【空城记：刘小东与鄂尔多斯】#正午故事#鄂尔多斯是个比较敏感的城市。其实政府也比想象的聪明，...,test,0,005EHRy1jw1ewv1azxwh1j30rs0lljzm,0,1. 信息来源为微博，这是一个广泛使用的社交媒体平台，可以发布真实的信息。\n2. 信息中提...,1,0,1. **事件**：图片显示了一群人在一个看起来像是城市边缘的空旷地带搭建某种结构，远处有高...,1,0,1. 图片来源\n- **水印或标识**：图片中有“@界面”的水印，表明这张图片可能来源于某...,1,0,合理性：鄂尔多斯曾被称作“鬼城”，因为其大量空置的建筑和低人口密度。然而，当地政府采取积极的...,1
7849,3895644894378737,【日本人是怎么拍黑帮电影的？】Tim Gallo出生于俄国，17岁后迁往日本生活。通过用镜头...,test,0,005EHRy1jw1ewtlhknf30j30jg0czq4y,1,1. 消息中提到的Tim Gallo是一个不太为人知的名字，而且他的背景（出生于俄国，后来迁...,0,1,1. **事件**：图片显示的是一名女性手持武器在街头，这与新闻中提到的“日本人是怎么拍黑帮...,0,0,1. 图片来源\n- **水印或标识**：图片中有“@界面”的水印，表明这张图片可能来源于某...,1,0,合理性：拍摄黑帮电影时，导演邀请真正的黑帮成员作为群众演员来增加影片的真实感，这种做法在日本...,1
7850,3893257346088858,#跟着面叔看世界#第一站，日本东京筑地市场。你现在到哪儿了？不会还堵在路上吧？一起来晒晒旅途...,test,0,005EHRy1jw1ewlzj6oenaj30qo0zkdm5,1,1. 信息来源的可信度：信息来源未被指明，无法验证信息的真实性和可靠性。\n2. 内容性质：...,0,0,1. **事件**：图片显示的是一个繁忙的市场场景，有许多摊位和顾客，这与新闻摘要中提到的日...,1,0,1. 图片来源\n- **水印或标识**：图片中有“@界面”的水印，表明这张图片可能来源于某...,1,1,可信度：该文本看起来像是社交媒体上的一个互动帖子，邀请用户分享他们在东京筑地市场的旅行照片。...,0
7851,3892828281041275,【现场图·广西柳城县连续发生多起爆炸】据@南宁电视台 ：网友称，现场听到几声巨响，车站，市场...,test,0,61e04755jw1ewkm8o7fyyj20c80gb0u2,0,1. 信息来源：该消息来自微博上的南宁电视台，微博作为一个公开平台，其发布的信息通常可以追溯...,1,0,1. **事件**：图片显示了一个城市的建筑物，背景中有一个巨大的烟雾柱，这与新闻摘要中提到...,1,0,1. 图片来源\n- **水印或标识**：图片中有“南宁电视台”的水印，表明这张图片可能来源...,1,1,可能性：同时在多个地点发生爆炸的情况较为罕见，除非是计划性的恐怖袭击或其他恶意行为。从描述来...,0


In [4]:

from Util import cal_rationale_metrics

for rationale_name in rationale_names:
    print(f'{rationale_name} metrics: \n '
          f'{cal_rationale_metrics(weibo_df[f"{rationale_name}_pred"].to_numpy(),weibo_df["label"].to_numpy())}')


td metrics: 
 {'acc': 0.8209601426206545, 'recall': 0.8131515273861729, 'recall_real': 0.7053816584294343, 'recall_fake': 0.9209213963429114, 'precision': 0.8401776032210654, 'precision_real': 0.8929440389294404, 'precision_fake': 0.7874111675126904, 'f1_macro': 0.8185534524033077, 'f1_real': 0.7881576928976837, 'f1_fake': 0.8489492119089317}
itc metrics: 
 {'acc': 0.7976569463899147, 'recall': 0.7986388907689401, 'recall_real': 0.8121911037891268, 'recall_fake': 0.7850866777487533, 'precision': 0.7976541022029819, 'precision_real': 0.7663212435233161, 'precision_fake': 0.8289869608826479, 'f1_macro': 0.797514759571859, 'f1_real': 0.7885897094108237, 'f1_fake': 0.8064398097328943}
img metrics: 
 {'acc': 0.5801604482363428, 'recall': 0.6031270771576913, 'recall_real': 0.9200988467874794, 'recall_fake': 0.28615530752790314, 'precision': 0.6813304968939999, 'precision_real': 0.5401353965183753, 'precision_fake': 0.8225255972696246, 'f1_macro': 0.5526386478620653, 'f1_real': 0.680682510664

In [6]:

from sklearn.model_selection import train_test_split


def split_data(df, train_ratio=0.8, valid_ratio=0.1, random_state=None, stratify_col=None):
    """
    将 pandas DataFrame 划分为训练集、验证集和测试集。

    参数:
        df (pd.DataFrame): 包含所有数据的 pandas DataFrame。
        train_ratio (float): 训练集所占比例，默认为 0.8。
        valid_ratio (float): 验证集所占比例，默认为 0.1。
        random_state (int, RandomState instance or None): 控制随机抽样的种子，默认为 None。
        stratify_col (str or None): 如果不为 None，则数据将按照指定列进行分层抽样，默认为 None。

    返回:
        tuple: 包含训练集、验证集和测试集的元组。
               每个集合都是原始 DataFrame 的一部分，包含所有原始列。
    """
    # 计算测试集的比例
    test_ratio = 1 - train_ratio - valid_ratio
    if test_ratio <= 0:
        raise ValueError("The sum of train_ratio and valid_ratio must be less than 1.")

    # 分层抽样的依据列（如果指定了）
    stratify = df[stratify_col] if stratify_col else None

    # 第一步：先从全部数据中划分出测试集
    remaining_df, test_df = train_test_split(
        df,
        test_size=test_ratio ,
        random_state=random_state,
        shuffle=True,
        stratify=stratify
    )

    # 更新分层抽样的依据列（如果指定了）
    stratify_remaining = remaining_df[stratify_col] if stratify_col else None

    # 第二步：从剩余的数据中划分出训练集和验证集
    train_df, valid_df = train_test_split(
        remaining_df,
        test_size=valid_ratio / (train_ratio + valid_ratio),
        random_state=random_state,
        shuffle=True,
        stratify=stratify_remaining
    )

    return train_df, valid_df, test_df

def count_dataset(df):
    return {
        'sum': df.shape[0],
        'real':(df['label']==label_str2int_dict['real']).sum(),
        'fake':(df['label']==label_str2int_dict['fake']).sum(),
    }

train_df,val_df,test_df = split_data(weibo_df,train_ratio=0.6,valid_ratio=0.2,random_state=666)
train_df['split'] = 'train'
val_df['split'] = 'val'
test_df['split'] = 'test'
print(f'train : {count_dataset(train_df)}')
print(f'val : {count_dataset(val_df)}')
print(f'test : {count_dataset(test_df)}')


train : {'sum': 4711, 'real': 2211, 'fake': 2500}
val : {'sum': 1571, 'real': 697, 'fake': 874}
test : {'sum': 1571, 'real': 734, 'fake': 837}


In [7]:
weibo_df.to_csv('weibo_llm_rationale.csv',index=False)
train_df.to_csv('train.csv',index=False)
val_df.to_csv('val.csv',index=False)
test_df.to_csv('test.csv',index=False)